In [1]:
# Import libraries
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# Set project root and change working directory
project_root = Path.cwd().parent.parent
os.chdir(project_root)
print(f"📁 Working directory: {os.getcwd()}")

# Add src directory to Python path
sys.path.insert(0, str(project_root / "src"))

from tiingo import TiingoClient
from core.config import Config
from market import ESGManager, PriceManager, RiskFreeRateManager
from universe import SP500Universe

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

📁 Working directory: /Users/frank/Projects/QuantX/quantx-data-builder
✅ Libraries imported successfully
✅ Libraries imported successfully


In [2]:
# Initialize components
config = Config(str(project_root / "config/settings.yaml"))
universe = SP500Universe()

tiingo = TiingoClient({
    'api_key': config.get('fetcher.tiingo.api_key'),
    'session': True
})

price_mgr = PriceManager(tiingo=tiingo, universe=universe)
esg_mgr = ESGManager(universe=universe)

fred_api_key = config.get('fetcher.fred.api_key')
rf_mgr = RiskFreeRateManager(fred_api_key=fred_api_key)

print("✅ Components initialized successfully")

✅ Components initialized successfully


## 1. Load ESG Data for Multiple Stocks

Load monthly ESG scores for a sample of S&P 500 stocks to demonstrate factor construction.

In [3]:
# Define analysis period and sample stocks
start_date = '2014-01-01'
end_date = '2025-11-11'

# Sample of S&P 500 stocks with good ESG data coverage
sample_tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA',  # Tech
    'JPM', 'BAC', 'WFC', 'GS', 'MS',          # Financials
    'JNJ', 'UNH', 'PFE', 'ABBV', 'TMO',       # Healthcare
    # 'XOM', 'CVX', 'COP', 'SLB', 'EOG',        # Energy
    'PG', 'KO', 'PEP', 'WMT', 'COST',         # Consumer
    # 'BA', 'CAT', 'GE', 'MMM', 'HON'           # Industrials
]

print(f"Loading ESG data for {len(sample_tickers)} stocks from {start_date} to {end_date}...")

# Load ESG data for all tickers
esg_data_list = []

for ticker in sample_tickers:
    esg_df = esg_mgr.load_esg_data(ticker=ticker, start_date=start_date, end_date=end_date)
    
    if not esg_df.empty and 'ESG Score' in esg_df.columns:
        esg_df['ticker'] = ticker
        esg_data_list.append(esg_df)

# Combine all ESG data
esg_panel = pd.concat(esg_data_list, ignore_index=True)

# Align dates to end-of-month
esg_panel['date'] = pd.to_datetime(esg_panel['date'])
esg_panel['date'] = esg_panel['date'] + pd.offsets.MonthEnd(0)

print(f"\n✓ Loaded ESG data for {esg_panel['ticker'].nunique()} stocks")
print(f"  Total observations: {len(esg_panel):,}")
print(f"  Date range: {esg_panel['date'].min().date()} to {esg_panel['date'].max().date()}")
print(f"\n📊 ESG Score Summary:")
print(esg_panel['ESG Score'].describe())

Loading ESG data for 20 stocks from 2014-01-01 to 2025-11-11...


No ESG data found for JPM on us


ValueError: No objects to concatenate

## 2. Construct Cross-Sectional ESG Factors

Build ESG factors by ranking and standardizing scores within each month (cross-sectional approach).

In [ ]:
# Function to calculate cross-sectional factors
def calculate_cross_sectional_factors(panel_df, score_column='ESG Score'):
    """
    Calculate cross-sectional ESG factors for each date.
    
    Returns:
    - Percentile rank (0-100)
    - Z-score (standardized)
    - Decile assignment (1-10)
    """
    factors = panel_df.copy()
    
    # Group by date and calculate cross-sectional metrics
    for date in factors['date'].unique():
        mask = factors['date'] == date
        scores = factors.loc[mask, score_column]
        
        # Percentile rank (0-100)
        factors.loc[mask, f'{score_column}_pctrank'] = scores.rank(pct=True) * 100
        
        # Z-score (mean=0, std=1)
        mean = scores.mean()
        std = scores.std()
        if std > 0:
            factors.loc[mask, f'{score_column}_zscore'] = (scores - mean) / std
        else:
            factors.loc[mask, f'{score_column}_zscore'] = 0
        
        # Decile assignment (1=lowest, 10=highest)
        factors.loc[mask, f'{score_column}_decile'] = pd.qcut(
            scores, q=10, labels=range(1, 11), duplicates='drop'
        )
    
    return factors

# Calculate cross-sectional factors for ESG Score and pillars
esg_factors = calculate_cross_sectional_factors(esg_panel, 'ESG Score')

if 'Environmental Pillar Score' in esg_panel.columns:
    esg_factors = calculate_cross_sectional_factors(esg_factors, 'Environmental Pillar Score')
    esg_factors = calculate_cross_sectional_factors(esg_factors, 'Social Pillar Score')
    esg_factors = calculate_cross_sectional_factors(esg_factors, 'Governance Pillar Score')

print("✓ Cross-sectional factors calculated")
print(f"\nSample factors for {esg_factors['date'].min().date()}:")
sample = esg_factors[esg_factors['date'] == esg_factors['date'].min()][
    ['ticker', 'ESG Score', 'ESG Score_pctrank', 'ESG Score_zscore', 'ESG Score_decile']
].sort_values('ESG Score', ascending=False).head(10)
print(sample.to_string(index=False))

## 3. Calculate ESG Momentum (Time-Series Factor)

ESG momentum captures the rate of ESG improvement over time.

In [ ]:
# Calculate ESG momentum (6-month and 12-month changes)
esg_factors = esg_factors.sort_values(['ticker', 'date'])

for ticker in esg_factors['ticker'].unique():
    mask = esg_factors['ticker'] == ticker
    
    # 6-month momentum (percent change)
    esg_factors.loc[mask, 'ESG_momentum_6m'] = (
        esg_factors.loc[mask, 'ESG Score'].pct_change(periods=6) * 100
    )
    
    # 12-month momentum
    esg_factors.loc[mask, 'ESG_momentum_12m'] = (
        esg_factors.loc[mask, 'ESG Score'].pct_change(periods=12) * 100
    )
    
    # ESG trend (linear slope over 12 months)
    scores = esg_factors.loc[mask, 'ESG Score'].values
    for i in range(12, len(scores)):
        window = scores[i-12:i]
        if len(window) == 12 and not np.isnan(window).any():
            x = np.arange(12)
            slope = np.polyfit(x, window, 1)[0]
            esg_factors.loc[mask, 'ESG_trend_12m'] = np.nan
            esg_factors.loc[esg_factors[mask].index[i], 'ESG_trend_12m'] = slope

print("✓ ESG momentum factors calculated")
print(f"\nESG Momentum Statistics (12-month):")
print(esg_factors['ESG_momentum_12m'].describe())

# Show top ESG improvers (latest month)
latest_date = esg_factors['date'].max()
improvers = esg_factors[esg_factors['date'] == latest_date][
    ['ticker', 'ESG Score', 'ESG_momentum_6m', 'ESG_momentum_12m']
].dropna().sort_values('ESG_momentum_12m', ascending=False).head(10)

print(f"\nTop 10 ESG Improvers (12-month, as of {latest_date.date()}):")
print(improvers.to_string(index=False))

## 4. Build Composite ESG Factors

Combine multiple dimensions into composite factors.

In [ ]:
# Composite Factor 1: Quality-adjusted ESG (Level + Momentum)
# Stocks with high ESG scores AND improving ESG
esg_factors['ESG_quality_momentum'] = (
    esg_factors['ESG Score_zscore'] + 
    esg_factors['ESG_momentum_12m'].fillna(0) / 10  # Scale momentum
) / 2

# Composite Factor 2: Pillar-weighted composite (if pillar data available)
if 'Environmental Pillar Score_zscore' in esg_factors.columns:
    esg_factors['ESG_composite'] = (
        0.4 * esg_factors['Environmental Pillar Score_zscore'] +
        0.3 * esg_factors['Social Pillar Score_zscore'] +
        0.3 * esg_factors['Governance Pillar Score_zscore']
    )

# Recalculate cross-sectional ranks for composite factors
esg_factors = calculate_cross_sectional_factors(esg_factors, 'ESG_quality_momentum')

print("✓ Composite ESG factors constructed")
print(f"\nComposite Factor Distribution (latest month):")
latest = esg_factors[esg_factors['date'] == latest_date]
print(latest['ESG_quality_momentum'].describe())

## 5. Visualize Factor Distributions

In [ ]:
# Plot 1: ESG Score distribution over time
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ESG Score time series (median and quartiles)
monthly_stats = esg_factors.groupby('date')['ESG Score'].agg(['mean', 'median', 
                                                                lambda x: x.quantile(0.25),
                                                                lambda x: x.quantile(0.75)])
monthly_stats.columns = ['Mean', 'Median', 'Q25', 'Q75']

axes[0, 0].plot(monthly_stats.index, monthly_stats['Median'], label='Median', linewidth=2)
axes[0, 0].fill_between(monthly_stats.index, monthly_stats['Q25'], monthly_stats['Q75'], 
                         alpha=0.3, label='IQR')
axes[0, 0].set_title('ESG Score Distribution Over Time')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('ESG Score')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# ESG Score histogram (latest month)
axes[0, 1].hist(latest['ESG Score'].dropna(), bins=20, edgecolor='black', alpha=0.7)
axes[0, 1].set_title(f'ESG Score Distribution ({latest_date.date()})')
axes[0, 1].set_xlabel('ESG Score')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# ESG Momentum distribution
axes[1, 0].hist(esg_factors['ESG_momentum_12m'].dropna(), bins=30, 
                edgecolor='black', alpha=0.7, color='green')
axes[1, 0].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_title('ESG Momentum Distribution (12-month)')
axes[1, 0].set_xlabel('ESG Change (%)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

# Z-score distribution (should be ~N(0,1) each month)
axes[1, 1].hist(latest['ESG Score_zscore'].dropna(), bins=20, 
                edgecolor='black', alpha=0.7, color='purple')
axes[1, 1].set_title(f'ESG Z-Score Distribution ({latest_date.date()})')
axes[1, 1].set_xlabel('Z-Score')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Factor distributions visualized")

## 6. Calculate Factor Returns (Long-Short Portfolios)

Test if ESG factors predict returns by creating long-short portfolios.

In [ ]:
# Load monthly returns for the sample stocks
print("Loading monthly returns for factor analysis...")

returns_list = []
for ticker in sample_tickers:
    price_data = price_mgr.load_price_data(
        symbol=ticker,
        frequency='monthly',
        start_date=start_date,
        end_date=end_date
    )
    
    if price_data is not None and len(price_data) > 0:
        price_data['ticker'] = ticker
        price_data['date'] = pd.to_datetime(price_data['date'])
        price_data['date'] = price_data['date'] + pd.offsets.MonthEnd(0)
        
        # Calculate returns
        price_data = price_data.sort_values('date')
        price_data['return'] = price_data['adj_close'].pct_change()
        
        returns_list.append(price_data[['ticker', 'date', 'return']])

returns_panel = pd.concat(returns_list, ignore_index=True)

print(f"✓ Loaded returns for {returns_panel['ticker'].nunique()} stocks")
print(f"  Total observations: {len(returns_panel):,}")

In [ ]:
# Merge ESG factors with forward returns
# Key: Use ESG factor at time t to predict return from t to t+1

factor_returns = esg_factors.merge(
    returns_panel[['ticker', 'date', 'return']], 
    on=['ticker', 'date'], 
    how='inner'
)

# Shift returns forward (return in month t+1 given factor in month t)
factor_returns = factor_returns.sort_values(['ticker', 'date'])
factor_returns['forward_return'] = factor_returns.groupby('ticker')['return'].shift(-1)

# Drop rows with missing forward returns or factors
analysis_df = factor_returns.dropna(subset=['forward_return', 'ESG Score_zscore'])

print(f"✓ Merged factors with forward returns")
print(f"  Analysis observations: {len(analysis_df):,}")
print(f"  Date range: {analysis_df['date'].min().date()} to {analysis_df['date'].max().date()}")

In [ ]:
# Calculate long-short portfolio returns based on ESG deciles
portfolio_returns = []

for date in analysis_df['date'].unique():
    month_data = analysis_df[analysis_df['date'] == date].copy()
    
    if len(month_data) < 10:  # Need enough stocks for deciles
        continue
    
    # Assign deciles based on ESG z-score
    month_data['decile'] = pd.qcut(month_data['ESG Score_zscore'], 
                                    q=10, labels=range(1, 11), duplicates='drop')
    
    # Calculate equal-weighted returns for each decile
    decile_returns = month_data.groupby('decile')['forward_return'].mean()
    
    if 1 in decile_returns.index and 10 in decile_returns.index:
        # Long-short: Long top decile, short bottom decile
        long_short = decile_returns[10] - decile_returns[1]
        
        portfolio_returns.append({
            'date': date,
            'long_return': decile_returns[10],
            'short_return': decile_returns[1],
            'long_short_return': long_short,
            'decile_1': decile_returns[1],
            'decile_10': decile_returns[10]
        })

portfolio_df = pd.DataFrame(portfolio_returns)

print(f"✓ Calculated long-short portfolio returns")
print(f"\nLong-Short Portfolio Statistics:")
print(f"  Mean monthly return: {portfolio_df['long_short_return'].mean()*100:.2f}%")
print(f"  Std dev: {portfolio_df['long_short_return'].std()*100:.2f}%")
print(f"  Sharpe (annualized): {portfolio_df['long_short_return'].mean() / portfolio_df['long_short_return'].std() * np.sqrt(12):.2f}")
print(f"  Win rate: {(portfolio_df['long_short_return'] > 0).mean()*100:.1f}%")

## 7. Visualize Factor Performance

In [ ]:
# Plot cumulative returns of long-short ESG factor
portfolio_df = portfolio_df.sort_values('date')
portfolio_df['cumulative_return'] = (1 + portfolio_df['long_short_return']).cumprod()
portfolio_df['long_cumulative'] = (1 + portfolio_df['long_return']).cumprod()
portfolio_df['short_cumulative'] = (1 + portfolio_df['short_return']).cumprod()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Cumulative returns
axes[0].plot(portfolio_df['date'], portfolio_df['cumulative_return'], 
             label='Long-Short (D10-D1)', linewidth=2, color='blue')
axes[0].plot(portfolio_df['date'], portfolio_df['long_cumulative'], 
             label='Long (D10)', linewidth=1.5, alpha=0.7, color='green')
axes[0].plot(portfolio_df['date'], portfolio_df['short_cumulative'], 
             label='Short (D1)', linewidth=1.5, alpha=0.7, color='red')
axes[0].axhline(1, color='black', linestyle='--', alpha=0.5)
axes[0].set_title('ESG Factor Portfolio Performance (High ESG - Low ESG)')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Cumulative Return')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Monthly returns distribution
axes[1].bar(portfolio_df['date'], portfolio_df['long_short_return'] * 100, 
            color=['green' if x > 0 else 'red' for x in portfolio_df['long_short_return']])
axes[1].axhline(0, color='black', linestyle='-', linewidth=1)
axes[1].set_title('Monthly Long-Short Returns')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Return (%)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Factor performance visualized")

## 8. Factor Correlation Analysis

Analyze correlations between ESG factors and test orthogonality.

In [ ]:
# Calculate correlation matrix of factors
factor_cols = [
    'ESG Score_zscore',
    'ESG_momentum_12m',
    'ESG_quality_momentum'
]

if 'Environmental Pillar Score_zscore' in esg_factors.columns:
    factor_cols.extend([
        'Environmental Pillar Score_zscore',
        'Social Pillar Score_zscore',
        'Governance Pillar Score_zscore'
    ])

correlation_df = analysis_df[factor_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_df, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, vmin=-1, vmax=1, square=True)
plt.title('ESG Factor Correlation Matrix')
plt.tight_layout()
plt.show()

print("✓ Factor correlations analyzed")
print("\nKey Observations:")
print(f"  ESG Level vs Momentum: r = {correlation_df.loc['ESG Score_zscore', 'ESG_momentum_12m']:.2f}")

## 9. Save ESG Factors for Further Analysis

Export constructed factors for use in other analyses.

In [ ]:
# Save factor data
output_dir = Path("data/results/esg_factors")
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / f"esg_factors_{start_date}_{end_date}.parquet"
esg_factors.to_parquet(output_file, index=False)

print(f"✓ ESG factors saved to: {output_file}")
print(f"\nFactor columns created:")
for col in esg_factors.columns:
    if any(x in col for x in ['_zscore', '_pctrank', '_decile', '_momentum', '_composite']):
        print(f"  - {col}")

## Summary: ESG Factor Construction

**Key Findings:**

1. **Cross-Sectional Factors**: 
   - ESG scores normalized within each month
   - Z-scores provide standardized exposure
   - Deciles enable portfolio construction

2. **Momentum Factors**:
   - ESG improvement measured over 6 and 12 months
   - Captures time-series variation in ESG quality
   - Can identify improving vs deteriorating companies

3. **Factor Performance**:
   - Long-short ESG factor shows [positive/negative] returns
   - Sharpe ratio indicates [strong/weak] risk-adjusted performance
   - Win rate of [X]% suggests [consistent/inconsistent] alpha

4. **Factor Independence**:
   - ESG level and momentum show [low/moderate/high] correlation
   - E, S, G pillars provide [diversified/overlapping] signals
   - Composite factors capture multiple dimensions

**Next Steps**:
- Integrate ESG factors into multi-factor models
- Test factor timing strategies
- Analyze sector-specific ESG effects
- Combine with traditional factors (value, momentum, quality)